In [128]:
import pandas as pd
import pandasql as ps
import numpy as np
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_summary import DataFrameSummary
from datetime import datetime

import sys
sys.path.append('../py')
from preprocess import process_portfolio, process_transcript

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [38]:
portfolio = pd.read_json('../data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/transcript.json', orient='records', lines=True)

In [39]:
portfolio = process_portfolio(portfolio)
transaction_transcript, offer_received_transcript, offer_viewed_transcript, offer_completed_transcript = process_transcript(transcript)

In [42]:
# オファーに有効期限をjoin
    offer_received_transcript_step1 = offer_received_transcript.merge(portfolio[['id', 'duration']], how='left', left_on='offer_id', right_on='id').drop('id', axis=1)

In [44]:
# 各オファーの有効期限を算出
offer_received_transcript_step1["offer_deadline"] = offer_received_transcript_step1["time"] + 24 * offer_received_transcript_step1["duration"]

In [48]:
# 各購入に関係する可能性のあるオファーをjoinする
transaction_transcript_step1 = transaction_transcript.drop('value', axis=1)
offer_received_transcript_step2 = offer_received_transcript_step1.drop('value', axis=1)

query = '''
    SELECT
        L.person,
        L.time,
        L.amount,
        R.time AS offer_received_at,
        R.offer_id,
        R.offer_deadline
    FROM
      transaction_transcript_step1 AS L
    LEFT JOIN
      offer_received_transcript_step2 AS R
    ON L.person = R.person
    AND L.time BETWEEN R.time AND R.offer_deadline
'''

transaction_transcript_step1 = ps.sqldf(query, locals())
transaction_transcript_step1.head()

,person,time,amount,offer_received_at,offer_id,offer_deadline
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,0.0,2906b810c7d4411798c6938adc9daaa5,168.0
2,54890f68699049c2a04d415abc25e717,0,13.23,NaN,None,NaN
3,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,0.0,5a8bc65990b245e5a138643cd4eb9837,72.0
4,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,0.0,fafdcd668e3743c1bb461111dcafc2a4,240.0


In [51]:
transaction_transcript_step1[transaction_transcript_step1.person == '9fa9ae8f57894cc9a3b8a9bbe0fc1b2f']

,person,time,amount,offer_received_at,offer_id,offer_deadline
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,0.0,2906b810c7d4411798c6938adc9daaa5,168.0
6112,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,42,21.55,0.0,2906b810c7d4411798c6938adc9daaa5,168.0
18280,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,114,32.87,0.0,2906b810c7d4411798c6938adc9daaa5,168.0
24288,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,156,19.99,0.0,2906b810c7d4411798c6938adc9daaa5,168.0
25926,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,168,20.97,0.0,2906b810c7d4411798c6938adc9daaa5,168.0
33714,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,204,31.09,NaN,None,NaN
62032,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,354,18.42,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,504.0
70456,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,390,29.71,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,504.0
71919,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,396,32.53,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,504.0
73335,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,402,33.36,336.0,2298d6c36e964ae4a3e7e9706d1fb8c2,504.0


In [56]:
# オファーが見られたか、見られた時間をjoin
offer_viewed_transcript_step1 = offer_viewed_transcript[['person', 'time', 'offer_id']]
offer_viewed_transcript_step1 = offer_viewed_transcript_step1.rename(columns={'time': 'offer_viewed_at'})

transaction_transcript_step2 = transaction_transcript_step1.merge(offer_viewed_transcript_step1, how='left', on=['person', 'offer_id'])
transaction_transcript_step2.head()

,person,time,amount,offer_received_at,offer_id,offer_deadline,offer_viewed_at
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,0.0,2906b810c7d4411798c6938adc9daaa5,168.0,0.0
2,54890f68699049c2a04d415abc25e717,0,13.23,NaN,None,NaN,NaN
3,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,0.0,5a8bc65990b245e5a138643cd4eb9837,72.0,66.0
4,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,0.0,fafdcd668e3743c1bb461111dcafc2a4,240.0,6.0


In [70]:
# オファーの影響による購入なのかフラグ付けする
query = '''
    SELECT
      *,
      CASE 
        WHEN offer_id IS NULL THEN 0
        WHEN offer_viewed_at IS NULL THEN 0
        WHEN time < offer_viewed_at THEN 0
        ELSE 1
      END AS is_influenced_by_offer
    FROM
      transaction_transcript_step2
'''

transaction_transcript_step3 = ps.sqldf(query, locals())
transaction_transcript_step3.head()

,person,time,amount,offer_received_at,offer_id,offer_deadline,offer_viewed_at,is_influenced_by_offer
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,0.0,2906b810c7d4411798c6938adc9daaa5,168.0,0.0,1
2,54890f68699049c2a04d415abc25e717,0,13.23,NaN,None,NaN,NaN,0
3,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,0.0,5a8bc65990b245e5a138643cd4eb9837,72.0,66.0,0
4,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,0.0,fafdcd668e3743c1bb461111dcafc2a4,240.0,6.0,0


In [100]:
transaction_transcript_step3[transaction_transcript_step3.person == '02c083884c7d45b39cc68e1314fec56c']

,person,time,amount,offer_received_at,offer_id,offer_deadline,offer_viewed_at,is_influenced_by_offer
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
774,02c083884c7d45b39cc68e1314fec56c,6,1.44,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
1764,02c083884c7d45b39cc68e1314fec56c,12,4.56,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
16830,02c083884c7d45b39cc68e1314fec56c,84,1.53,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
18129,02c083884c7d45b39cc68e1314fec56c,90,0.50,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
20622,02c083884c7d45b39cc68e1314fec56c,102,9.80,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
26261,02c083884c7d45b39cc68e1314fec56c,132,4.10,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
28188,02c083884c7d45b39cc68e1314fec56c,144,2.73,0.0,ae264e3637204a6fb9bb56bc8210ddfd,168.0,0.0,1
40134,02c083884c7d45b39cc68e1314fec56c,198,1.22,NaN,None,NaN,NaN,0
98488,02c083884c7d45b39cc68e1314fec56c,420,3.18,408.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,648.0,NaN,0


In [101]:
offer_related_transaction = transaction_transcript_step3[transaction_transcript_step3.is_influenced_by_offer == 1]
offer_not_related_transaction = transaction_transcript_step3[transaction_transcript_step3.is_influenced_by_offer == 0]

In [103]:
transaction_transcript_step4 = offer_related_transaction.groupby(['person', 'offer_id'])['amount'].sum().reset_index()
transaction_transcript_step4.head()

,person,offer_id,amount
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,8.57
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,22.16
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,14.11
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,82.76
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,45.08


In [104]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [105]:
portfolio.head()

,difficulty,duration,id,offer_type,reward,num_channel,channel_web,channel_email,channel_mobile,channel_social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,3,0,1,1,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,4,1,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0,3,1,1,1,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,3,1,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,2,1,1,0,0


In [106]:
transaction_transcript_step5 = transaction_transcript_step4.merge(profile, how='left', left_on='person', right_on='id').drop('id', axis=1)
transaction_transcript_step5 = transaction_transcript_step5.merge(portfolio, how='left', left_on='offer_id', right_on='id').drop('id', axis=1)
transaction_transcript_step5.head()

,person,offer_id,amount,age,became_member_on,gender,income,difficulty,duration,offer_type,reward,num_channel,channel_web,channel_email,channel_mobile,channel_social
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,8.57,33,20170421,M,72000.0,0,4,informational,0,3,1,1,1,0
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,22.16,33,20170421,M,72000.0,0,3,informational,0,3,0,1,1,1
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,14.11,33,20170421,M,72000.0,5,5,bogo,5,4,1,1,1,1
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,82.76,33,20170421,M,72000.0,10,10,discount,2,4,1,1,1,1
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,45.08,40,20180109,O,57000.0,20,10,discount,5,2,1,1,0,0


In [121]:
offer_not_related_transaction_step1 = offer_not_related_transaction[['person', 'amount']]
aggregation = {
    'amount': ['min', 'max', 'sum', 'mean', 'count']
}

offer_not_related_transaction_step1= offer_not_related_transaction_step1.groupby('person')['amount'].agg(aggregation).reset_index()
offer_not_related_transaction_step1.head()

person amount                               
                                       min    max    sum       mean count
0  0009655768c64bdeb2e877511632db8f   8.57  28.16  91.88  15.313333     6
1  00116118485d4dfda04fdbaba9a87b5c   0.20   3.19   4.09   1.363333     3
2  0011e0d4e6b944f998e987f904e8c1e5  13.49  13.49  13.49  13.490000     1
3  0020c2b971eb4e9188eac86d93036a77  17.24  33.86  98.53  24.632500     4
4  0020ccbbb6d84e358d3414a3ff76cffd  16.27  16.27  16.27  16.270000     1

In [122]:
offer_not_related_transaction_step1.columns = ['_'.join(col) for col in offer_not_related_transaction_step1.columns]
offer_not_related_transaction_step1.head()

,person_,amount_min,amount_max,amount_sum,amount_mean,amount_count
0,0009655768c64bdeb2e877511632db8f,8.57,28.16,91.88,15.313333,6
1,00116118485d4dfda04fdbaba9a87b5c,0.20,3.19,4.09,1.363333,3
2,0011e0d4e6b944f998e987f904e8c1e5,13.49,13.49,13.49,13.490000,1
3,0020c2b971eb4e9188eac86d93036a77,17.24,33.86,98.53,24.632500,4
4,0020ccbbb6d84e358d3414a3ff76cffd,16.27,16.27,16.27,16.270000,1


In [125]:
transaction_transcript_step6 = transaction_transcript_step5.merge(offer_not_related_transaction_step1, how='left', left_on='person', right_on='person_').drop('person_', axis=1)
transaction_transcript_step6.head()

,person,offer_id,amount,age,became_member_on,gender,income,difficulty,duration,offer_type,...,num_channel,channel_web,channel_email,channel_mobile,channel_social,amount_min,amount_max,amount_sum,amount_mean,amount_count
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,8.57,33,20170421,M,72000.0,0,4,informational,...,3,1,1,1,0,8.57,28.16,91.88,15.313333,6.0
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,22.16,33,20170421,M,72000.0,0,3,informational,...,3,0,1,1,1,8.57,28.16,91.88,15.313333,6.0
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,14.11,33,20170421,M,72000.0,5,5,bogo,...,4,1,1,1,1,8.57,28.16,91.88,15.313333,6.0
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,82.76,33,20170421,M,72000.0,10,10,discount,...,4,1,1,1,1,8.57,28.16,91.88,15.313333,6.0
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,45.08,40,20180109,O,57000.0,20,10,discount,...,2,1,1,0,0,13.49,13.49,13.49,13.490000,1.0


In [138]:
transaction_transcript_step7 = transaction_transcript_step6.copy(deep=True)
transaction_transcript_step7['became_member_on'] = pd.to_datetime(transaction_transcript_step7.became_member_on.map(lambda x: str(x)), format='%Y%m%d')
transaction_transcript_step7['registar_year'] = transaction_transcript_step7['became_member_on'].dt.year
transaction_transcript_step7['registar_month'] = transaction_transcript_step7['became_member_on'].dt.month
transaction_transcript_step7 = pd.get_dummies(transaction_transcript_step7, columns=['gender', 'offer_type'], dummy_na=True)
transaction_transcript_step7 = transaction_transcript_step7.drop(['became_member_on', 'offer_type_nan'], axis=1)
transaction_transcript_step7.head()

,person,offer_id,amount,age,income,difficulty,duration,reward,num_channel,channel_web,...,amount_count,registar_year,registar_month,gender_F,gender_M,gender_O,gender_nan,offer_type_bogo,offer_type_discount,offer_type_informational
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,8.57,33,72000.0,0,4,0,3,1,...,6.0,2017,4,0,1,0,0,0,0,1
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,22.16,33,72000.0,0,3,0,3,0,...,6.0,2017,4,0,1,0,0,0,0,1
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,14.11,33,72000.0,5,5,5,4,1,...,6.0,2017,4,0,1,0,0,1,0,0
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,82.76,33,72000.0,10,10,2,4,1,...,6.0,2017,4,0,1,0,0,0,1,0
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,45.08,40,57000.0,20,10,5,2,1,...,1.0,2018,1,0,0,1,0,0,1,0


後はどのくらいの精度が出るか。。。

In [139]:
len(transaction_transcript_step7)

38363

In [140]:
len(transaction_transcript_step7[transaction_transcript_step7.age == 118])

4738

In [143]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split

In [149]:
profile_null_records = transaction_transcript_step7[transaction_transcript_step7.age == 118]
transaction_transcript_step8 = transaction_transcript_step7[transaction_transcript_step7.age != 118]
transaction_transcript_step8 = transaction_transcript_step8.fillna(0)

In [150]:
X = transaction_transcript_step8.drop(['person', 'offer_id', 'amount'], axis=1)
y = transaction_transcript_step8['amount']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [165]:
model = RandomForestRegressor(max_depth=10, n_estimators=100)
scores = cross_val_score(model, X_train, y_train, cv=5)
print(scores)

[0.12488623 0.11056142 0.20425029 0.170486   0.0136042 ]


In [166]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [172]:
importance_df = pd.DataFrame({'name':X.columns, 'importance': model.feature_importances_})
importance_df.sort_values(by='importance', ascending=False)

,name,importance
12,amount_sum,0.342078
11,amount_max,0.105273
13,amount_mean,0.098031
0,age,0.062674
1,income,0.061362
10,amount_min,0.056105
3,duration,0.046847
16,registar_month,0.034989
14,amount_count,0.033518
15,registar_year,0.026998


オファーのタイプによってモデルを分けたほうが良いかも